# Setting up Paths

In [1]:
#path to the data
pathCOMPASOutput   = '/home/cneijssel/Documents/Projects/Data/CosmicInt/'
#path to compasrepo which contains ther popsynth folder
pathPostProcessing = '/home/cneijssel/Documents/COMPASpop/popsynth/Papers/NeijsselEtAL/PostProcessing'



# Imports

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import h5py as h5
import time
import sys
import copy
#Quick fudge to make import from ../Scripts work
import sys


# Custom Imports from COMPASrepo

In [3]:
sys.path.append(pathPostProcessing + '/2_CosmicIntegration')
#Given settings and redshifts returns rates (2D arrays) Loads the data
import ClassCosmicIntegrator  as CI 
# Quickly set ticks and labels in a nice preset font
import AxesLayout             as Layout


# The cosmic integrator class

In [4]:
#Will only look at BBHs so might as well set everything
minz = 0.
maxz = 1.
resz = 100
Data = CI.CosmicIntegrator(COMPASpath = pathCOMPASOutput, DCOtypes='BBH',\
       minRedshift=minz,   maxRedshift=maxz, nrRedshiftBins=resz, optimistic=False)


Remember to  self.setCOMPASDCOmask() and self.setCOMPASData()


# The plot

In [5]:
#The distribution of DCOs as a function of redshift

method     = ['logNormal'                ,'MZ_GSMF'                     ,'MZ_GSMF'                      , 'MZ_GSMF']
SFRs       = ['Neijssel et al. (2019)'   ,'Madau et al. (2014)'         ,'Madau et al. (2014)'          , 'Madau et al. (2014)']
logNormal  = ['Neijssel Phenomenological', None                         , None                          , None]
GSMFs      = [None                       , 'Panter et al. (2004) Single', 'Furlong et al. (2015) Single', 'Panter et al. (2004) Single']
MZs        = [None                       , 'Langer et al. (2006)'       , 'Langer et al. (2006)'        , 'Ma et al. (2015)']

linestyles = ['-'                        , '--'                         , ':'                           , '-.']
labels     = ['Preferred model'          , 'Barrett et al. (2018) MSSFR', 'Furlong et al. (2015) Single', 'Ma et al. (2015)']


cm       = plt.get_cmap('viridis')
nColors  = 3+1  #want three colours for three variations (1 used in other plot later)
                    # dont want the brigthest colour hence I add a dummy value.
                    #Should be consistent with figure of metallicity PDFs
mycolors = [cm(x) for x in np.linspace(0,1 , nColors)]

colours  =  [ 'k'                        ,mycolors[0]                   , mycolors[1]                   , mycolors[2]]




rates    = []
totals   = []
for nrL, L in enumerate(labels):
    Data.MSSFR.Zprescription         = method[nrL]
    Data.MSSFR.SFRprescription       = SFRs[nrL]
    Data.MSSFR.logNormalPrescription = logNormal[nrL]
    Data.MSSFR.GSMFprescription      = GSMFs[nrL]
    Data.MSSFR.ZMprescription        = MZs[nrL]
    Data.cosmologicalIntegration()
        
    
    Row        =np.sum(Data.PerSystemPerRedshift_ratesObserved, axis=0)
    rates.append(Row)
    totals.extend([np.sum(Row)])


In [6]:
fig, axes        = plt.subplots(1,1,figsize=(9,8))

############
#  Top panel with MSSFR
############
Mtot       = Data.COMPAS.mChirp
bins       = np.linspace(0,60,100)
center     = (bins[1:]+bins[:-1])/2.
dMtot      = np.diff(bins)

for nrL, L in enumerate(labels):
    rate       = rates[nrL]
    rate, _    = np.histogram(Mtot, bins=bins, weights = rate)
    rate       = np.divide(rate, dMtot)
    rate       = rate / np.sum(rate)
    label      = labels[nrL]+r' $\rm \normalfont [%s \ yr^{-1}]$' %(int(totals[nrL]))
    axes.plot(center, rate, lw=3.5, c=colours[nrL], label=label, linestyle=linestyles[nrL], alpha=0.85)
nameX = r'$\rm M_{chirp}\ \ [M_{\odot}] $'
nameY = r'$\rm dP / dM_{chirp} \ [M_{\odot}^{-1}]  $'
axes = Layout.layoutAxes(axes, nameX=nameX, nameY=nameY)
axes.legend(loc=1, prop={'size':16})
axes.set_ylim(top=0.07)


plt.tight_layout()
plt.subplots_adjust(wspace=0., hspace=0.0)
plt.savefig('./MassDistributionAtRedshiftObs.png', dpi=300, transparent=True)
#plt.show()
plt.close()